If you want to be in the `PedestrianAvoidancePOMDP` environment, run the following cell. If the package has been added to the main julia environment, it is not necessary.

In [1]:
] activate ../ 

In [2]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using PedestrianAvoidancePOMDP
using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/xubuntu/.julia/compiled/v1.0/PedestrianAvoidancePOMDP/UuJ2S.ji for PedestrianAvoidancePOMDP [a80ff0fe-e6a0-11e8-3b05-15413bd5ea7a]
└ @ Base loading.jl:1187
┌ Warning: Package PedestrianAvoidancePOMDP does not have EmergencyBrakingSystem in its dependencies:
│ - If you have PedestrianAvoidancePOMDP checked out for development and have
│   added EmergencyBrakingSystem as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PedestrianAvoidancePOMDP
└ Loading EmergencyBrakingSystem into PedestrianAvoidancePOMDP from project dependency, future warnings for PedestrianAvoidancePOMDP are suppressed.


In [3]:
policy = load("../policy/policy.jld2")["policy"]; # need to be updated

In [4]:
pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)

parameters = CrosswalkParams()
parameters.roadway_length = 400.0
parameters.obstacles_visible = true

obstacle_offset = -2
obstacle_1 = ConvexPolygon([VecE2(34, obstacle_offset), VecE2(34, obstacle_offset-3), VecE2(46.5, obstacle_offset-3), VecE2(46.5, obstacle_offset)],4)
obstacle_2 = ConvexPolygon([VecE2(34, +4.5), VecE2(34, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
parameters.obstacles = [obstacle_1, obstacle_2]
#parameters.obstacles = [obstacle_2]

env = CrosswalkEnv(parameters)
pomdp.env = env


ego_id = 1
ped_id = 2
ped2_id = 3


ego_v = 11.11
ped_v = 5.4/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -7.0
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;

if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -2.0
    ped_v = 0.0
end

ped_y_start = -3.
ped_v = 1.0
ego_y = 0.0
# Car definition
ego_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
ego = Vehicle(ego_initial_state, VehicleDef(), 1)

# Pedestrian definition 
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

ped2_v = 0.0
ped2 = Vehicle(VehicleState(VecSE2(54., 5., 0.78), env.crosswalk, env.roadway, ped2_v), AutomotivePOMDPs.PEDESTRIAN_DEF, ped2_id)


scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



cam = FitToContentCamera(0.);
timestep = 0.2


pos_noise = 0.001
vel_noise = 0.000
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 


# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

pomdp.desired_velocity = ego_v
pomdp.ΔT = timestep
models[ego_id] = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),
    env=env,
    sensor=sensor,
    obstacles=env.obstacles,
    timestep=timestep,
    pomdp=pomdp,
    policy=policy,
    updater=updater,
    ego_vehicle=ego,
    desired_velocity=ego_v,
    b=PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, ego_y, ego_v)
)

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.0001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.0001) # dumb model



nticks = 50
rec = SceneRecord(nticks+1, timestep)

risk = Float64[]
sensor_observations = [Vehicle[]]
belief = Dict{Int64, SingleOCFBelief}[]
ego_vehicle = Vehicle[]
action_pomdp = SingleOCFAction[]

obs_callback = (ObservationCallback(risk,sensor_observations, belief, ego_vehicle, action_pomdp),)

@elapsed begin
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
end




--------------------------POMDP high level planner----------------------- t: 0.0
EGO: x/y:-3.846666666666664 / 0.0 v: 11.11
Vehicle(2, VehicleState(VecSE2({50.000, -3.000}, 1.571), Frenet(RoadIndex({1, 0.350000}, {2, 1}), 7.000, 150.000, 0.000), 1.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.000}, 0.780), Frenet(RoadIndex({1, 0.750000}, {2, 1}), 15.000, 146.000, -0.791), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
b-length: 28
SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 12.0, 50.0, -4.0, 1.57, 0.0], [0.0, 12.0, 50.0, -3.0, 1.57, 0.0], [0.0, 12.0, 50.0, -4.0, 1.57, 1.0], [0.0, 12.0, 50.0, -3.0, 1.57, 1.0], [0.0, 12.0, 50.0, -4.0, 1.57, 2.0], [0.0, 12.0, 50.0, -3.0, 1.57, 2.0], [0.0, 12.0, -10.0, -10.0, 0.0, 0.0]], [0.129032, 0.129032, 0.129032, 0.129032, 0.129032, 0.129032, 0.225806])
action combined: [-4.0, 0.0]
-> perfect observation: [0.0, 11.11, 47.5, -3.0, 1.5708, 1.0]
--------------------------POMDP high level

b-length: 30
b-length: 38
SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[1.0, 12.0, 38.0, -4.0, 1.57, 0.0], [1.0, 12.0, 40.0, -4.0, 1.57, 0.0], [1.0, 12.0, 42.0, -4.0, 1.57, 0.0], [1.0, 12.0, 44.0, -4.0, 1.57, 0.0], [1.0, 12.0, 46.0, -4.0, 1.57, 0.0], [1.0, 12.0, 48.0, -4.0, 1.57, 0.0], [1.0, 12.0, 38.0, -4.0, 1.57, 1.0], [1.0, 12.0, 40.0, -4.0, 1.57, 1.0], [1.0, 12.0, 42.0, -4.0, 1.57, 1.0], [1.0, 12.0, 44.0, -4.0, 1.57, 1.0], [1.0, 12.0, 46.0, -4.0, 1.57, 1.0], [1.0, 12.0, 48.0, -4.0, 1.57, 1.0], [1.0, 12.0, 38.0, -4.0, 1.57, 2.0], [1.0, 12.0, 40.0, -4.0, 1.57, 2.0], [1.0, 12.0, 42.0, -4.0, 1.57, 2.0], [1.0, 12.0, 44.0, -4.0, 1.57, 2.0], [1.0, 12.0, 46.0, -4.0, 1.57, 2.0], [1.0, 12.0, 48.0, -4.0, 1.57, 2.0], [1.0, 12.0, -10.0, -10.0, 0.0, 0.0]], [0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.043956, 0.208791])
action combined: [-4.0, 0.0]


b-length: 84
b-length: 98
SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[1.0, 10.0, 26.0, -4.0, 1.57, 0.0], [1.0, 10.0, 28.0, -4.0, 1.57, 0.0], [1.0, 10.0, 30.0, -4.0, 1.57, 0.0], [1.0, 10.0, 32.0, -4.0, 1.57, 0.0], [1.0, 10.0, 26.0, -4.0, 1.57, 1.0], [1.0, 10.0, 28.0, -4.0, 1.57, 1.0], [1.0, 10.0, 30.0, -4.0, 1.57, 1.0], [1.0, 10.0, 32.0, -4.0, 1.57, 1.0], [1.0, 10.0, 26.0, -4.0, 1.57, 2.0], [1.0, 10.0, 28.0, -4.0, 1.57, 2.0], [1.0, 10.0, 30.0, -4.0, 1.57, 2.0], [1.0, 10.0, 32.0, -4.0, 1.57, 2.0], [1.0, 10.0, -10.0, -10.0, 0.0, 0.0]], [0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.0655738, 0.213115])
action combined: [1.0, 0.0]
-> perfect observation: [1.0, 9.91, 27.2647, -1.7, 1.5708, 1.0]
--------------------------POMDP high level planner----------------------- t: 2.4
EGO: x/y:22.177333333333337 / 1.0 v: 9.509999999999998
Vehicle(2, VehicleState(VecSE2({50.000, -0.500}, 1.571), Frenet(R

b-length: 100
b-length: 81
SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[1.0, 7.0, 12.0, -4.0, 1.57, 0.0], [1.0, 7.0, 14.0, -4.0, 1.57, 0.0], [1.0, 7.0, 12.0, -4.0, 1.57, 1.0], [1.0, 7.0, 14.0, -4.0, 1.57, 1.0], [1.0, 7.0, 12.0, -4.0, 1.57, 2.0], [1.0, 7.0, 14.0, -4.0, 1.57, 2.0], [1.0, 7.0, -10.0, -10.0, 0.0, 0.0]], [0.129032, 0.129032, 0.129032, 0.129032, 0.129032, 0.129032, 0.225806])
action combined: [-4.0, 0.0]
-> perfect observation: [1.0, 6.71, 13.9687, -0.4, 1.5708, 1.5]
--------------------------POMDP high level planner----------------------- t: 4.000000000000001
EGO: x/y:34.83333333333334 / 1.0 v: 6.309999999999995
Vehicle(2, VehicleState(VecSE2({50.000, 0.900}, 1.571), Frenet(RoadIndex({1, 0.545000}, {2, 1}), 10.900, 150.000, 0.000), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.141}, 0.780), Frenet(RoadIndex({1, 0.757036}, {2, 1}), 15.141, 146.000, -0.791), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
b-length

b-length: 14
b-length: 25
SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[1.0, 6.0, -10.0, -10.0, 0.0, 0.0]], [1.0])
action combined: [-4.0, 0.0]
-> perfect observation: [1.0, 5.11, 3.58867, 1.7, 1.5708, 1.0]
--------------------------POMDP high level planner----------------------- t: 6.000000000000003
EGO: x/y:44.95333333333336 / 1.0 v: 5.309999999999994
Vehicle(2, VehicleState(VecSE2({50.000, 2.900}, 1.571), Frenet(RoadIndex({1, 0.645000}, {2, 1}), 12.900, 150.000, 0.000), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.281}, 0.780), Frenet(RoadIndex({1, 0.764072}, {2, 1}), 15.281, 146.000, -0.791), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
b-length: 37
b-length: 57
SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[1.0, 6.0, -10.0, -10.0, 0.0, 0.0]], [1.0])
action combined: [-4.0, 0.0]
-> perfect observation: [1.0, 5.31, 2.54667, 1.9, 1.5708, 0.5]
--------------------------POMDP high level planner------

Vehicle(2, VehicleState(VecSE2({50.000, 6.500}, 1.571), Frenet(RoadIndex({1, 0.825000}, {2, 1}), 16.500, 150.000, 0.000), 1.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.000}, 0.780), Frenet(RoadIndex({1, 0.750001}, {2, 1}), 15.000, 146.000, -0.791), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[1.0, 9.0, -10.0, -10.0, 0.0, 0.0]], [1.0])
action combined: [1.0, 0.0]
--------------------------POMDP high level planner----------------------- t: 9.799999999999997
EGO: x/y:72.35133333333337 / 1.0 v: 9.109999999999992
Vehicle(2, VehicleState(VecSE2({50.000, 6.700}, 1.571), Frenet(RoadIndex({1, 0.835000}, {2, 1}), 16.700, 150.000, 0.000), 1.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.000}, 0.780), Frenet(RoadIndex({1, 0.750003}, {2, 1}), 15.000, 146.000, -0.791), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
SparseCat{Array{SingleOCFState,1},Arra

37.341551543

In [7]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, CarFollowCamera(2,10.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("pomdp.mp4", film) # Write to a webm video

Frames{MIME{Symbol("image/png")}}("/tmp/tmpgiyFWO", 0x0000000000000033, 5.0, nothing)